### Initialise workspace from Azure ML workspace

In [1]:
from azureml.core import Workspace

source_dir = "./source_dir/"
ws = Workspace.from_config(source_dir + "config.json")
print(ws)

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Interactive authentication successfully completed.
Performing interactive authentication. Please follow the instructions on the terminal.


You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


AuthenticationException: AuthenticationException:
	Message: Could not retrieve user token. Please run 'az login'
	InnerException It is required that you pass in a value for the "algorithms" argument when calling decode().
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Could not retrieve user token. Please run 'az login'"
    }
}

In [ ]:
from azureml.core.model import Model

# Register model
model = Model.register(workspace = ws, 
                       model_name = "Commonlit-RoBERTa-Base",
                       model_path = "./outputs/Commonlit-RoBERTa-Base",
                       model_framework = "Custom")

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment(name = "Commonlit-Bi-LSTM")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")

conda_dep.add_pip_package("transformers)

conda_dep.add_pip_package("azureml-defaults")

env.python.conda_dependencies = conda_dep

In [ ]:
inference_config = InferenceConfig(
    environment = env,
    source_directory = "./source_dir",
    entry_script = "./RoBERTa_score.py",
)

In [ ]:
from azureml.core.webservice import AciWebservice
aci_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2)
service = Model.deploy(
    ws,
    "commonlit-roberta-base",
    [model],
    inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)